In [35]:
import pandas as pd
import numpy as np

In [36]:
#load the data
data = pd.read_csv("data/July17.csv")
data.head()

,ORIGIN,DEST,DEP_DELAY,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 9
0,ABE,DTW,-5.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ABE,DTW,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ABE,DTW,-9.0,-9.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ABE,DTW,-8.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ABE,DTW,-6.0,-19.0,NaN,NaN,NaN,NaN,NaN,NaN


var cities =
{"nodes": [{"name":"Chicago"}],
"links":[{"source":1,"target":0,"value":20}]};

In [37]:
def getCityData(city, dataset):
    city_destin_data = dataset[dataset.DEST == city]
    city_destin_data = city_destin_data.groupby('ORIGIN', as_index = False).mean()
    city_origin_data = dataset[dataset.ORIGIN == city]
    city_origin_data = city_origin_data.groupby('DEST', as_index = False).mean()
    return city_destin_data, city_origin_data

In [38]:
def dictPad(text):
    return dict({"name":text})

def getNodes(city, incoming, outgoing):
    cities = []
    for i in incoming.ORIGIN:
        cities.append(i)
    for i in outgoing.DEST:
        cities.append(i)
    cities = list(set(cities))
    return len(cities), [city]+cities+cities

def getNewNodes(city, incoming, outgoing):
    in_cities = []
    out_cities = []
    for i in incoming.ORIGIN:
        in_cities.append(i)
    for i in outgoing.DEST:
        out_cities.append(i)
    return len(in_cities), [city]+in_cities+out_cities

In [39]:
def getIncomingLinks(data, nodes, city):
    links = []
    for index, row in data.iterrows():
        links.append(dict({"source": nodes.index(row['ORIGIN']) ,"target":nodes.index(city) ,"value":row['ARR_DELAY'] }))
    return links

def getOutgoingLinks(data, nodes, city, num):
    links = []
    for index, row in data.iterrows():
        links.append(dict({"source": nodes.index(city),"target": max(loc for loc, val in enumerate(nodes) if val == row['DEST']) ,"value":row['DEP_DELAY'] }))
    return links


In [42]:
import json

def visualize(cities, nodes):
    for idx,l in enumerate(cities['links']):
        print (1+idx, nodes[l['source']] + "-->" + nodes[l['target']] + "=" + str(l['value']))

def getSankeyInfo(city):
    incoming, outgoing = getCityData(city, data)

    incoming = incoming.sort_values('ARR_DELAY', axis=0, ascending=False)
    outgoing = outgoing.sort_values('DEP_DELAY', axis=0, ascending=False)

    num, nodes = getNewNodes(city, incoming, outgoing)

    links_in = getIncomingLinks(incoming, nodes, city)
    links_out = getOutgoingLinks(outgoing, nodes, city, num)
    links = links_in + links_out
    
    cities = {"nodes": list(map(lambda x:dictPad(x),nodes)), "links": links}
    
    visualize(cities,nodes)
    
    file = 'data/sankey/data_{0}.json'.format(city)
    with open(file, 'w') as outfile:
        json.dump(cities, outfile)


In [43]:
list_of_cities = list(set(list(data.ORIGIN)+list(data.DEST)))

for c in list_of_cities:
    getSankeyInfo(c)

1 ATL-->OAJ=34.26865671641791
2 OAJ-->ATL=25.0
1 MYR-->ACY=34.58620689655172
2 RSW-->ACY=33.96666666666667
3 BOS-->ACY=27.870967741935484
4 FLL-->ACY=26.23728813559322
5 ATL-->ACY=15.03225806451613
6 MCO-->ACY=13.056818181818182
7 TPA-->ACY=-14.928571428571429
8 ACY-->BOS=51.03333333333333
9 ACY-->MCO=18.0561797752809
10 ACY-->RSW=17.566666666666666
11 ACY-->FLL=7.60655737704918
12 ACY-->ATL=-2.838709677419355
13 ACY-->MYR=-4.0
14 ACY-->TPA=-4.857142857142857
1 ORD-->BOI=9.931034482758621
2 SMF-->BOI=8.741935483870968
3 GEG-->BOI=7.52
4 LAS-->BOI=7.3283582089552235
5 OAK-->BOI=5.193548387096774
6 DFW-->BOI=2.903225806451613
7 PDX-->BOI=1.2419354838709677
8 SFO-->BOI=0.18333333333333332
9 DEN-->BOI=0.0967741935483871
10 PHX-->BOI=0.05434782608695652
11 MDW-->BOI=-2.225806451612903
12 SLC-->BOI=-2.523529411764706
13 SEA-->BOI=-5.077669902912621
14 MSP-->BOI=-5.517647058823529
15 LAX-->BOI=-9.67741935483871
16 SAN-->BOI=-13.225806451612904
17 SJC-->BOI=-23.6
18 BOI-->LAS=12.83582089552238

1 PDX-->SNA=7.239130434782608
2 OAK-->SNA=5.380281690140845
3 LAS-->SNA=4.661290322580645
4 SFO-->SNA=4.442013129102845
5 SEA-->SNA=4.112745098039215
6 SJC-->SNA=3.786127167630058
7 PHX-->SNA=3.2492668621700878
8 ORD-->SNA=2.5130434782608697
9 SMF-->SNA=0.7319587628865979
10 MDW-->SNA=0.34615384615384615
11 DFW-->SNA=-0.7
12 DEN-->SNA=-2.981012658227848
13 ATL-->SNA=-4.0
14 SLC-->SNA=-4.7785714285714285
15 IAH-->SNA=-5.535714285714286
16 DAL-->SNA=-5.615384615384615
17 HOU-->SNA=-6.290322580645161
18 EWR-->SNA=-11.013698630136986
19 MSP-->SNA=-18.363636363636363
20 SNA-->SLC=14.843971631205674
21 SNA-->OAK=11.516279069767442
22 SNA-->ATL=9.96774193548387
23 SNA-->SFO=9.596491228070175
24 SNA-->SMF=8.564766839378239
25 SNA-->PHX=8.336257309941521
26 SNA-->LAS=8.321428571428571
27 SNA-->DEN=7.70125786163522
28 SNA-->HOU=6.290322580645161
29 SNA-->PDX=6.032608695652174
30 SNA-->SJC=5.256484149855908
31 SNA-->ORD=3.432900432900433
32 SNA-->MDW=2.6538461538461537
33 SNA-->DAL=1.884615384615

1 LEX-->MSP=34.0
2 MLI-->MSP=32.91304347826087
3 EWR-->MSP=22.553719008264462
4 PIT-->MSP=21.569620253164558
5 IAH-->MSP=21.185185185185187
6 ORF-->MSP=21.166666666666668
7 DSM-->MSP=18.103448275862068
8 CVG-->MSP=13.454545454545455
9 HIB-->MSP=13.365384615384615
10 MDW-->MSP=11.91864406779661
11 TPA-->MSP=11.451612903225806
12 ORD-->MSP=11.376223776223776
13 DAY-->MSP=10.454545454545455
14 RIC-->MSP=9.258064516129032
15 FAR-->MSP=9.237623762376238
16 BOS-->MSP=8.5
17 JFK-->MSP=8.295652173913043
18 MSN-->MSP=7.794871794871795
19 CLT-->MSP=7.362445414847161
20 IAD-->MSP=7.184782608695652
21 JAX-->MSP=6.838709677419355
22 MCO-->MSP=6.47682119205298
23 BWI-->MSP=6.3478260869565215
24 DFW-->MSP=5.464968152866242
25 RST-->MSP=5.240384615384615
26 IDA-->MSP=5.2
27 DCA-->MSP=4.906976744186046
28 ABR-->MSP=4.475409836065574
29 ICT-->MSP=4.391304347826087
30 ROC-->MSP=4.387096774193548
31 LAS-->MSP=4.223744292237443
32 FSD-->MSP=4.0
33 XNA-->MSP=3.6666666666666665
34 ATL-->MSP=3.658767772511848

1 DEN-->LAR=17.384615384615383
2 LAR-->DEN=13.634615384615385
1 ORD-->SPI=23.493827160493826
2 SPI-->ORD=29.426829268292682
1 ANC-->BRW=0.8823529411764706
2 SCC-->BRW=-5.35
3 BRW-->ANC=-9.604166666666666
4 BRW-->FAI=-10.482758620689655
1 DEN-->JMS=10.415094339622641
2 DVL-->JMS=-9.451612903225806
3 JMS-->DVL=7.0754716981132075
4 JMS-->DEN=2.870967741935484
1 MIA-->STL=27.576271186440678
2 CVG-->STL=23.0625
3 LGA-->STL=21.369230769230768
4 EWR-->STL=19.568421052631578
5 TPA-->STL=18.76923076923077
6 ORD-->STL=17.258928571428573
7 LAS-->STL=16.047619047619047
8 ATL-->STL=15.746594005449591
9 DFW-->STL=15.436893203883495
10 SLC-->STL=14.564516129032258
11 DSM-->STL=14.052631578947368
12 LAX-->STL=13.453947368421053
13 AUS-->STL=13.421052631578947
14 SAN-->STL=12.985074626865671
15 CLT-->STL=11.789473684210526
16 PNS-->STL=11.4
17 ICT-->STL=10.12280701754386
18 RSW-->STL=9.26086956521739
19 HOU-->STL=9.033898305084746
20 MSY-->STL=8.982142857142858
21 FLL-->STL=8.341463414634147
22 CMH-->S

1 PHX-->SAF=3.6451612903225805
2 DFW-->SAF=1.7916666666666667
3 SAF-->DFW=8.221052631578948
4 SAF-->PHX=1.3870967741935485
1 SLC-->WYS=10.283333333333333
2 WYS-->SLC=9.64516129032258
1 EWR-->CLE=25.506666666666668
2 FLL-->CLE=25.338709677419356
3 RDU-->CLE=22.580645161290324
4 IAD-->CLE=21.71875
5 DFW-->CLE=18.382978723404257
6 SFO-->CLE=15.22972972972973
7 BOS-->CLE=13.96774193548387
8 CLT-->CLE=13.866666666666667
9 ATL-->CLE=13.823899371069182
10 LGA-->CLE=13.358407079646017
11 ORD-->CLE=11.08296943231441
12 STL-->CLE=10.727272727272727
13 BWI-->CLE=10.714285714285714
14 MCO-->CLE=9.017391304347827
15 DTW-->CLE=6.714285714285714
16 BNA-->CLE=6.511363636363637
17 MDW-->CLE=6.196629213483146
18 PHX-->CLE=6.041666666666667
19 LAS-->CLE=5.284403669724771
20 MSY-->CLE=3.7241379310344827
21 LAX-->CLE=2.69811320754717
22 PDX-->CLE=-2.6153846153846154
23 DCA-->CLE=-2.6198347107438016
24 TPA-->CLE=-2.870967741935484
25 IAH-->CLE=-2.9482758620689653
26 MSP-->CLE=-3.4948453608247423
27 DEN-->CL

...

1 ATL-->LFT=4.921739130434783
2 IAH-->LFT=0.25
3 LFT-->ATL=6.568965517241379
4 LFT-->IAH=0.3194444444444444
1 EWR-->SMF=73.06451612903226
2 JFK-->SMF=68.16666666666667
3 IAD-->SMF=24.548387096774192
4 ORD-->SMF=20.0561797752809
5 LGB-->SMF=16.20967741935484
6 CLT-->SMF=14.625
7 DFW-->SMF=11.815068493150685
8 SFO-->SMF=11.741935483870968
9 BOS-->SMF=11.590909090909092
10 LAS-->SMF=7.791304347826087
11 PHX-->SMF=7.740259740259741
12 PDX-->SMF=7.373239436619718
13 SEA-->SMF=5.430523917995444
14 IAH-->SMF=5.392156862745098
15 ONT-->SMF=4.825136612021858
16 SLC-->SMF=4.202312138728324
17 SNA-->SMF=4.119170984455959
18 LAX-->SMF=2.8098159509202456
19 BOI-->SMF=2.6129032258064515
20 DEN-->SMF=2.5342465753424657
21 DAL-->SMF=2.3870967741935485
22 BUR-->SMF=2.189054726368159
23 MDW-->SMF=2.1846153846153844
24 SAN-->SMF=1.1466346153846154
25 BWI-->SMF=-1.7096774193548387
26 HNL-->SMF=-2.3225806451612905
27 ATL-->SMF=-6.976470588235294
28 OGG-->SMF=-14.833333333333334
29 MSP-->SMF=-15.96470588235

23 LGA-->TPA=9.942528735632184
24 BDL-->TPA=9.782608695652174
25 ISP-->TPA=9.56140350877193
26 LAX-->TPA=8.979166666666666
27 RDU-->TPA=8.819277108433734
28 BWI-->TPA=8.2578125
29 IAH-->TPA=7.965116279069767
30 DEN-->TPA=6.446236559139785
31 PHL-->TPA=6.036842105263158
32 BNA-->TPA=5.868852459016393
33 MSY-->TPA=5.636363636363637
34 SFO-->TPA=5.354838709677419
35 PVD-->TPA=5.298245614035087
36 BOS-->TPA=4.987261146496815
37 IND-->TPA=3.7572815533980584
38 CVG-->TPA=3.5555555555555554
39 PIT-->TPA=2.4788732394366195
40 SAT-->TPA=1.4
41 MKE-->TPA=-0.06779661016949153
42 BHM-->TPA=-0.2857142857142857
43 STL-->TPA=-1.121212121212121
44 CLE-->TPA=-1.7419354838709677
45 ALB-->TPA=-2.838709677419355
46 TTN-->TPA=-3.611111111111111
47 ROC-->TPA=-4.838709677419355
48 DTW-->TPA=-6.124087591240876
49 SDF-->TPA=-7.2
50 MHT-->TPA=-8.055555555555555
51 MSP-->TPA=-11.360655737704919
52 ACY-->TPA=-12.0
53 SEA-->TPA=-12.903225806451612
54 TPA-->TTN=58.111111111111114
55 TPA-->HPN=40.903225806451616
56 

1 DTW-->ITH=-3.2705882352941176
2 ITH-->DTW=-0.18823529411764706
1 GUC-->IAH=-6.0
1 SFO-->MFR=9.016129032258064
2 PHX-->MFR=0.8387096774193549
3 LAX-->MFR=-0.25806451612903225
4 DEN-->MFR=-3.9464285714285716
5 SLC-->MFR=-7.629032258064516
6 MFR-->SFO=7.951612903225806
7 MFR-->PHX=7.466666666666667
8 MFR-->LAX=4.0
9 MFR-->DEN=1.7321428571428572
10 MFR-->SLC=-6.32258064516129
1 DEN-->MTJ=-0.09090909090909091
2 IAH-->MTJ=-11.258064516129032
3 MTJ-->IAH=0.6451612903225806
4 MTJ-->DEN=-1.8181818181818181
1 IAH-->BRO=-2.5
2 BRO-->IAH=-5.087378640776699
1 FSD-->LAS=39.333333333333336
2 FLL-->LAS=24.043478260869566
3 BOS-->LAS=20.20353982300885
4 PHL-->LAS=18.10576923076923
5 MIA-->LAS=16.440860215053764
6 HNL-->LAS=13.025316455696203
7 CLT-->LAS=12.96
8 ONT-->LAS=12.902439024390244
9 BUR-->LAS=12.591549295774648
10 EWR-->LAS=12.53072625698324
11 SFO-->LAS=11.064156206415621
12 DFW-->LAS=11.055837563451776
13 JFK-->LAS=10.366925064599483
14 PHX-->LAS=10.207423580786026
15 ORD-->LAS=10.07451923

1 ANC-->FAI=-1.141304347826087
2 ORD-->FAI=-1.5483870967741935
3 SEA-->FAI=-6.423913043478261
4 BRW-->FAI=-7.620689655172414
5 MSP-->FAI=-13.258064516129032
6 FAI-->ORD=11.266666666666667
7 FAI-->MSP=5.838709677419355
8 FAI-->SEA=-2.8
9 FAI-->ANC=-5.59349593495935
1 ORD-->MKG=15.418181818181818
2 MKG-->ORD=23.2
1 ORD-->MCI=31.36279069767442
2 LGA-->MCI=25.011363636363637
3 DFW-->MCI=14.78688524590164
4 CVG-->MCI=13.578947368421053
5 FLL-->MCI=13.35483870967742
6 LAS-->MCI=11.986013986013987
7 ATL-->MCI=10.314363143631436
8 LAX-->MCI=9.426229508196721
9 DAL-->MCI=8.913705583756345
10 IND-->MCI=8.714285714285714
11 AUS-->MCI=8.419354838709678
12 OAK-->MCI=8.16
13 DEN-->MCI=8.09966777408638
14 STL-->MCI=7.4907407407407405
15 ABQ-->MCI=7.032258064516129
16 PHX-->MCI=6.677631578947368
17 DCA-->MCI=5.491228070175438
18 PHL-->MCI=5.461538461538462
19 IAH-->MCI=5.376470588235295
20 MDW-->MCI=4.948616600790514
21 SFO-->MCI=4.19672131147541
22 SAT-->MCI=3.4516129032258065
23 TPA-->MCI=3.08928571

1 SFO-->FCA=60.516129032258064
2 DEN-->FCA=3.6333333333333333
3 ORD-->FCA=1.7428571428571429
4 SLC-->FCA=1.4787234042553192
5 MSP-->FCA=-3.2686567164179103
6 ATL-->FCA=-4.0
7 LAX-->FCA=-13.0
8 FCA-->SFO=62.29032258064516
9 FCA-->ATL=9.11111111111111
10 FCA-->ORD=5.6571428571428575
11 FCA-->DEN=5.233333333333333
12 FCA-->SLC=2.3191489361702127
13 FCA-->MSP=-0.08955223880597014
14 FCA-->LAX=-0.3333333333333333
1 JFK-->PSE=41.851851851851855
2 MCO-->PSE=26.24137931034483
3 PSE-->JFK=30.25925925925926
4 PSE-->MCO=19.46551724137931
1 BHM-->BWI=24.75862068965517
2 GRR-->BWI=20.566666666666666
3 CVG-->BWI=20.40566037735849
4 MEM-->BWI=19.56140350877193
5 LAS-->BWI=17.57847533632287
6 IND-->BWI=17.49411764705882
7 MCI-->BWI=16.695652173913043
8 MDW-->BWI=15.694174757281553
9 ECP-->BWI=15.583333333333334
10 CLT-->BWI=14.609154929577464
11 ABQ-->BWI=14.298507462686567
12 CLE-->BWI=14.118181818181819
13 STL-->BWI=13.737704918032787
14 ALB-->BWI=13.67251461988304
15 ORD-->BWI=12.54679802955665
16 

In [69]:
# newlist = sorted(sorted(links, key=lambda m: m['target']) , key=lambda k: k['value']) 
# newlist

In [70]:
# cities = {"nodes": list(map(lambda x:dictPad(x),nodes)), "links": newlist}
# import json
# file = 'data_{0}.txt'.format(city+"sort")
# with open(file, 'w') as outfile:
#     json.dump(cities, outfile)

1 SJC-->JFK=73.48387096774194
2 SMF-->JFK=58.241379310344826
3 DCA-->JFK=50.516129032258064
4 PHL-->JFK=50.193548387096776
5 RSW-->JFK=44.18032786885246
6 FLL-->JFK=39.91694352159468
7 ORD-->JFK=38.89041095890411
8 RDU-->JFK=38.63793103448276
9 HOU-->JFK=37.357142857142854
10 MCO-->JFK=32.174107142857146
11 PSE-->JFK=30.0
12 MIA-->JFK=29.924342105263158
13 CLT-->JFK=29.91891891891892
14 PBI-->JFK=28.5
15 ACK-->JFK=27.77777777777778
16 BUF-->JFK=27.663366336633665
17 JAX-->JFK=26.65909090909091
18 MVY-->JFK=25.63157894736842
19 SAV-->JFK=25.17543859649123
20 BGR-->JFK=22.727272727272727
21 LGB-->JFK=22.666666666666668
22 ATL-->JFK=21.826291079812208
23 ROC-->JFK=21.261904761904763
24 BUR-->JFK=20.774193548387096
25 ABQ-->JFK=19.612903225806452
26 PWM-->JFK=19.36521739130435
27 BOS-->JFK=19.26843657817109
28 SJU-->JFK=19.13372093023256
29 TPA-->JFK=19.085106382978722
30 LAS-->JFK=17.885416666666668
31 IAD-->JFK=17.8625
32 AUS-->JFK=17.30635838150289
33 SFO-->JFK=16.989583333333332
34 MSY